# Imports

In [ ]:
%env OE_LICENSE=/Users/alexpayne/oe_license.txt


In [ ]:
from openeye import oechem
from openeye import oedepict
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import importlib
from covid_moonshot_ml.docking import docking as d
from covid_moonshot_ml.data import openeye as oe
from covid_moonshot_ml import schema
from covid_moonshot_ml.datasets import utils
import yaml

importlib.reload(d)
importlib.reload(schema)
importlib.reload(utils)

In [ ]:
with open("../data/cmpd_to_frag.yaml") as f:
    cmpd_to_frag_dict = yaml.safe_load(f)

# Load SARS Re-docking results

In [ ]:
fn = "/Volumes/Rohirrim/local_test/datasets/sars-redocked.csv"

In [ ]:
df = pd.read_csv(fn).fillna("-1")

In [ ]:
df.head()

In [ ]:
df[df["Crystal_ID"] == "Mpro-P2028"]["Chemgauss4"]

In [ ]:
vector_list = [df[df.Crystal_ID == crystal_id]["Chemgauss4"] for crystal_id in set(df.Crystal_ID)]

In [ ]:
np.shape(vector_list)

In [ ]:
len(vector_list)

In [ ]:
vector_list[219]

## oh right....not all of the structures have been docked to all of the compounds

## how many have?

In [ ]:
len(set(df.Crystal_ID))

### there are 220 unique crystal ids

In [ ]:
compound_list = np.array(list(set(df.Compound_ID)))

In [ ]:
compound_list

In [ ]:
structures_per_compound_list = [df[df.Compound_ID == compound_id]["Crystal_ID"] for compound_id in compound_list]

In [ ]:
np.intersect1d(structures_per_compound_list[0], structures_per_compound_list[0])

In [ ]:
matrix = [len(np.intersect1d(target, test)) 
          for target in structures_per_compound_list 
          for test in structures_per_compound_list]

In [ ]:
matrix[0:5]

In [ ]:
np.shape(matrix)

In [ ]:
npmatrix = np.ndarray(buffer=np.array(matrix), shape=(220,220), dtype=int)

In [ ]:
npmatrix

In [ ]:
kmeans = KMeans(n_clusters=6, random_state=0).fit(npmatrix)

In [ ]:
len(kmeans.labels_)

In [ ]:
kmeans.labels_

In [ ]:
len(compound_list[kmeans.labels_ == 0])

In [ ]:
compound_list[kmeans.labels_ == 0]

In [ ]:
len(compound_list[kmeans.labels_ == 1])

In [ ]:
compound_list[kmeans.labels_ == 1]

In [ ]:
compound_list[kmeans.labels_ == 2]

In [ ]:
compound_list[kmeans.labels_ == 3]

In [ ]:
compound_list[kmeans.labels_ == 4]

In [ ]:
compound_list[kmeans.labels_ == 5]

# Get molecule smiles

In [ ]:
cmpd_tracker = "/Users/alexpayne/Scientific_Projects/mers-drug-discovery/"
xtal_compounds = d.parse_xtal(cmpd_tracker, "")
cmpd_df = pd.read_csv(cmpd_tracker)

In [ ]:
cmpd_df.head()

In [ ]:
cmpd_df.index = cmpd_df["Compound ID"]

In [ ]:
smiles = cmpd_df.loc[compound_list[kmeans.labels_ == 5][0], "SMILES"]

In [ ]:
smiles

# Write out cluster results

In [ ]:
for cluster in range(0,5):
    filtered_cmpd_ids = compound_list[kmeans.labels_ == cluster]
    for i in range(len(filtered_cmpd_ids)):
        cmpd_id = filtered_cmpd_ids[i]
        smiles = cmpd_df.loc[cmpd_id, "SMILES"]
        if type(smiles) == str:
            mol = oechem.OEGraphMol()
            oechem.OESmilesToMol(mol, smiles)
            oe.write_openeye_ligand(mol, 
                                out_fn=f"/Volumes/Rohirrim/local_test/chemiinformatics/cluster{cluster}_{cmpd_id}.png")

# How well do clusters overlap with series assignments?

In [ ]:
compound_list[kmeans.labels_ == 0]

In [ ]:
cmpd_df.loc[cmpd_id, "Moonshot Series"]

In [ ]:
for i in range(0,6):
    for cmpd_id in compound_list[kmeans.labels_ == 0]:
        print(cmpd_df.loc[cmpd_id, "Moonshot Series"])